Wprowadzenie do Exact Cover Problemu

Stworzenie instancji problemu Exact Cover:

In [10]:
A = {1,4,7}
B = {1,4}
C = {4,5,7}
D = {3,5,6}
E = {2,3,6,7}
F = {2,7}
routes = [A,B,C,D,E,F]

Stworzenie obiektu ExactCoverSolver:

In [11]:
from utils import EXACTCOVER
exact_cover_solver = EXACTCOVER(routes, p = 2)


In [12]:
exact_cover_solver.solve()

  0%|          | 0/100 [00:00<?, ?it/s]